In [6]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
from tensorflow_hub.module_v2 import load

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')
print(tf.__version__)

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)


2.9.0


In [7]:
train_ds = object_detector.DataLoader.from_pascal_voc(
    './project-6-at-2023-04-30-00-12-74270469/train',
    './project-6-at-2023-04-30-00-12-74270469/train',
    ['Counted_Objects', 'Not_Counted_Objects']
)

val_ds = object_detector.DataLoader.from_pascal_voc(
    './project-6-at-2023-04-30-00-12-74270469/validate',
    './project-6-at-2023-04-30-00-12-74270469/validate',
    ['Counted_Objects', 'Not_Counted_Objects']
)

In [8]:
print(train_ds.size)
print(val_ds.size)

108
27


In [9]:
#spec = model_spec.get('efficientdet_lite4')

spec = object_detector.EfficientDetSpec(
  model_name='efficientdet-lite4', 
  uri='https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/2', 
  hparams={'max_instances_per_image': 8000})

In [10]:
model = object_detector.create(train_ds, model_spec=spec, batch_size=4, train_whole_model=True, epochs=200, validation_data=val_ds)

Epoch 1/200
27/27 [==============================] - 314s 10s/step - det_loss: 1.8001 - cls_loss: 1.1155 - box_loss: 0.0137 - reg_l2_loss: 0.1078 - loss: 1.9079 - learning_rate: 0.0065 - gradient_norm: 0.8780 - val_det_loss: 1.6009 - val_cls_loss: 1.0637 - val_box_loss: 0.0107 - val_reg_l2_loss: 0.1078 - val_loss: 1.7087
Epoch 2/200
27/27 [==============================] - 255s 9s/step - det_loss: 1.4672 - cls_loss: 0.9717 - box_loss: 0.0099 - reg_l2_loss: 0.1078 - loss: 1.5750 - learning_rate: 0.0050 - gradient_norm: 1.8338 - val_det_loss: 1.2040 - val_cls_loss: 0.7384 - val_box_loss: 0.0093 - val_reg_l2_loss: 0.1078 - val_loss: 1.3118
Epoch 3/200
27/27 [==============================] - 256s 10s/step - det_loss: 1.0645 - cls_loss: 0.6587 - box_loss: 0.0081 - reg_l2_loss: 0.1078 - loss: 1.1723 - learning_rate: 0.0050 - gradient_norm: 2.8678 - val_det_loss: 1.0302 - val_cls_loss: 0.6440 - val_box_loss: 0.0077 - val_reg_l2_loss: 0.1078 - val_loss: 1.1380
Epoch 4/200
27/27 [=============

In [11]:
model.evaluate(train_ds)

2/2 [==============================] - 65s 24s/step



{'AP': 0.64636254,
 'AP50': 0.84137005,
 'AP75': 0.7656196,
 'APs': 0.0,
 'APm': 0.019130128,
 'APl': 0.661737,
 'ARmax1': 0.18152286,
 'ARmax10': 0.49664366,
 'ARmax100': 0.7023425,
 'ARs': 0.0,
 'ARm': 0.10526316,
 'ARl': 0.71798486,
 'AP_/Counted_Objects': 0.60403186,
 'AP_/Not_Counted_Objects': 0.6886932}

In [12]:
model.evaluate(val_ds)

1/1 [==============================] - 18s 18s/step



{'AP': 0.44786686,
 'AP50': 0.64125055,
 'AP75': 0.5253297,
 'APs': 0.0,
 'APm': 0.08514851,
 'APl': 0.4511773,
 'ARmax1': 0.13704027,
 'ARmax10': 0.3800764,
 'ARmax100': 0.5499466,
 'ARs': 0.0,
 'ARm': 0.08181818,
 'ARl': 0.55375,
 'AP_/Counted_Objects': 0.4000126,
 'AP_/Not_Counted_Objects': 0.4957211}

In [13]:
model.export(export_dir='../obj_model_training/',tflite_filename='efd_4_uc_ha_bs4.tflite')